# 3 eme Version - Amélioration du cadre d'analyse

In [7]:
import numpy as np
import random 
import simpy
from scipy.stats import chi2

L'objectif est d'améliorer le cadre de simulation de la version précédente en y implémentant un niveau d'aléatoiriété supplémentaire. En effet il faut implémenter le fait que le nombre de panneaux pouvant proposer des bid requests est variable et évolue dynamiquement. Ainsi, le délai entre chaque bid request ne suit plus seulement une loi à paramètre constant mais à paramètre variable. L'intérêt est ainsi de simuler le fait qu'à certaines heures de la journée le délai entre deux br est beaucoup plus court que d'autres heures.
    
Le second objectif est de pouvoir simuler ce qu'il s'est passé sur un jour, puis une semaine puis un mois par exemple et stocker les résultats dans un fichier (csv ou json par exemple) afin de réutiliser ces résultats dans la mise en place d'un algorithme de pacing plus élaboré que précédemment.

### Etape 1 - Générer les fonctions des variables aléatoires ainsi que les constantes

In [5]:
# Constantes globales
nombre_jours = 1 
nombre_panneaux = 10000

In [29]:
np.random.normal(loc=0.8, scale=0.1, size=10)

array([0.83642402, 0.94099984, 0.7705799 , 0.88506604, 0.80623922,
       0.89490328, 0.70311218, 0.81182573, 0.8721371 , 0.79208179])

In [15]:
def delai(lam):
    secondes = np.random.poisson(lam)
    
    #Simuler une proba de pb technique (2% de chance d'en avoir un)
    if not random.random() < 0.98:
        secondes = np.random.poisson(600)
    return secondes

In [3]:
def imps():
    nb_imp = np.random.poisson(3)
    return nb_imp

In [ ]:
def bid_proba(p):
    return random.random() < p

In [ ]:
def open_rtb(env):
    B = 30
    duree_totale = 60
    cible = B/duree_totale
    while True:
        # Nombre d'impressions
        nb_imp = imps()
        print(f"bid request avec {nb_imp} impressions")
        number_imps_br.append(nb_imp)
        
        # Calcul du budget par seconde restant
        rt = duree_totale - env.now
        B_sec = B/rt
        b_sec.append(B_sec)
        print(f"\t Budget par sec de {B_sec} euros")
        
        # Proba d'acheter
        if B_sec >= cible:
            proba = 1
        else:
            proba = 0
        
        # Achat ou pas
        if bid_proba(proba):
            B -= 1
            print(f"\t Achat de {P} euros. Il reste {B} euros")
        else:
            print(f"\t Pas d'achat. Il reste {B} euros")
        B_remain.append(B)
        
        # Temps restant de la simulation
        print(f"\t Il reste {rt} secondes avant la fin de la simulation \n")
        
        # Temps avant la prochaine BR
        time = delai() 
        
        # Test pour faire des trucs a la fin de la simulation
        if rt < time:
            print("Fin de la simulation")
            print(f"Le budget restant est de {B} euros")
        yield env.timeout(time)